# Collaborative Filtering

In [6]:
# for data manipulation
import numpy as np
import pandas as pd
import os

# use surprise for collaborative filtering
from surprise import Reader, Dataset
from surprise import SVD, evaluate
from surprise import NMF

## Read in data

#### Game data

In [7]:
game_data_path = "data/final_game_data/"
files = os.listdir(game_data_path)
season = pd.read_csv(game_data_path + files[0])

In [8]:
season.shape

(2632, 98)

#### Odds data

In [10]:
odds_data_path = "data/odds_data_processed/"
odds_files = os.listdir(odds_data_path)

In [11]:
odds = pd.read_csv(odds_data_path + odds_files[1])

In [12]:
odds.shape

(1288, 14)

#### Reconciling names

In [13]:
season_names = {'Golden State Warriors':'GSW',
                'Los Angeles Lakers': 'LAL',
                'San Antonio Spurs': 'SAS',
                'Cleveland Cavaliers': 'CLE',
                'Denver Nuggets': 'DEN',
                'Indiana Pacers': 'IND',
                'Memphis Grizzlies': 'MEM',
                'New Jersey Nets': 'BRK',
                'Brooklyn Nets': 'BRK',
                'New Orleans Hornets': 'NOP',
                'New Orleans Pelicans': 'NOP',
                'Orlando Magic': 'ORL',
                'Toronto Raptors': 'TOR',
                'Miami Heat': 'MIA',
                'Seattle SuperSonics': 'SEA',
                'Utah Jazz': 'UTA',
                'Atlanta Hawks': 'ATL',
                'Boston Celtics': 'BOS',
                'Charlotte Bobcats': 'CHA',
                'Charlotte Hornets': 'CHA',
                'Chicago Bulls': 'CHI',
                'Los Angeles Clippers': 'LAC',
                'Minnesota Timberwolves': 'MIN',
                'Phoenix Suns': 'PHO',
                'Dallas Mavericks': 'DAL',
                'Houston Rockets': 'HOU',
                'Milwaukee Bucks': 'MIL',
                'Philadelphia 76ers': 'PHI',
                'Washington Wizards': 'WAS',
                'Detroit Pistons': 'DET',
                'New York Knicks': 'NYK',
                'Sacramento Kings': 'SAC',
                'Portland Trail Blazers': 'POR',
                'Oklahoma City Thunder': 'OKC'
        }
odds_names = {}
for name in list(pd.unique(odds.Home)):
    found = False
    for s_name in season_names:
        if name in s_name:
            found = True
            odds_names[name] = season_names[s_name]
    if not found: print(name)
odds_names["LA Lakers"] = "LAL"
odds_names["LA Clippers"] = "LAC"
odds_names["Okla City"] = "OKC"

LA Lakers
LA Clippers


In [14]:
odds["Home"] = odds["Home"].apply(lambda x: odds_names[x])
odds["Away"] = odds["Away"].apply(lambda x: odds_names[x])

season["team"] = season["team"].apply(lambda x: season_names[x])
season["opponent"] = season["opponent"].apply(lambda x: season_names[x])

#### Merging the two data tables

In [15]:
def make_index(row, col1, col2, col3):
    return str(row[col1]) + str(row[col2]) + str(row[col3])

season["date"] = season["date"].apply(lambda x: str(x)[:-1])

season["Index"] = season.apply(lambda x: make_index(x, "date", "team", "opponent"), axis=1)

odds["Date"] = odds["Date"].apply(lambda x: "".join(x.split("-")))

odds["Index"] = odds.apply(lambda x: make_index(x, "Date", "Home", "Away"), axis=1)

merged = pd.merge(odds, season, on='Index')

merged = merged.drop(["Unnamed: 0_x", "Unnamed: 0_y", "date", "Home", "Away", "index"], axis = 1)

In [16]:
merged.sample(1)

,Date,OU,Spread,OU_2H,Spread_2H,ML_home,ML_away,Points,Win Margin,2H Points,...,opponent_HOB,opponent_STL,opponent_TRB,opponent_FTA,opponent_BLK,opponent_FTr,opponent_TS%,opponent_FT/FGA,opponent_3P%,home
67,20071109,201.0,8.5,97.5,7.0,315,-375,207,-5,92,...,1.564103,12.0,43.0,15.0,5.0,0.169,0.554393,0.146067,0.454545,1


In [20]:
data = merged.set_index("Index")
#data = data.drop(["index"], axis = 1)
data = data.sort_index()

In [21]:
data.shape

(1285, 106)

In [23]:
data.head(5)

,Date,OU,Spread,OU_2H,Spread_2H,ML_home,ML_away,Points,Win Margin,2H Points,...,opponent_HOB,opponent_STL,opponent_TRB,opponent_FTA,opponent_BLK,opponent_FTr,opponent_TS%,opponent_FT/FGA,opponent_3P%,home
Index,,,,,,,,,,,,,,,,,,,,,
20071030GSWUTA,20071030,214.5,3.0,105.5,3.0,-120,100,213,-21,100,...,1.585366,9.0,56.0,36.0,7.0,0.400,0.552721,0.333333,0.454545,1
20071030LALHOU,20071030,191.0,2.5,99,3,190,-230,188,-2,102,...,1.676471,10.0,49.0,31.0,5.0,0.419,0.541990,0.283784,0.272727,1
20071030SASPOR,20071030,184.0,12.5,95,5,-1400,900,203,9,95,...,1.384615,1.0,40.0,17.0,4.0,0.218,0.567384,0.166667,0.461538,1
20071031BRKCHI,20071031,188.0,2.0,94,3,-125,105,215,9,95,...,1.605263,7.0,45.0,26.0,3.0,0.271,0.479337,0.197917,0.347826,1
20071031CLEDAL,20071031,186.0,3.0,91.5,4.0,120,-140,166,-18,78,...,1.696970,6.0,36.0,20.0,8.0,0.290,0.591260,0.246377,0.500000,1


In [28]:
dates = pd.unique(data.Date)
prev_games = 3

In [34]:
for i, date in enumerate(dates):
    if i < prev_games: continue
    
    window = dates[i - prev_games:i]
    
    window_data = data.loc[data['Date'].isin(window)]
    
    break

In [35]:
window_data

,Date,OU,Spread,OU_2H,Spread_2H,ML_home,ML_away,Points,Win Margin,2H Points,...,opponent_HOB,opponent_STL,opponent_TRB,opponent_FTA,opponent_BLK,opponent_FTr,opponent_TS%,opponent_FT/FGA,opponent_3P%,home
Index,,,,,,,,,,,,,,,,,,,,,
20071030GSWUTA,20071030,214.5,3.0,105.5,3.0,-120,100,213,-21,100,...,1.585366,9.0,56.0,36.0,7.0,0.400,0.552721,0.333333,0.454545,1
20071030LALHOU,20071030,191.0,2.5,99,3,190,-230,188,-2,102,...,1.676471,10.0,49.0,31.0,5.0,0.419,0.541990,0.283784,0.272727,1
20071030SASPOR,20071030,184.0,12.5,95,5,-1400,900,203,9,95,...,1.384615,1.0,40.0,17.0,4.0,0.218,0.567384,0.166667,0.461538,1
20071031BRKCHI,20071031,188.0,2.0,94,3,-125,105,215,9,95,...,1.605263,7.0,45.0,26.0,3.0,0.271,0.479337,0.197917,0.347826,1
20071031CLEDAL,20071031,186.0,3.0,91.5,4.0,120,-140,166,-18,78,...,1.696970,6.0,36.0,20.0,8.0,0.290,0.591260,0.246377,0.500000,1
20071031DENSEA,20071031,213.0,10.5,107.5,6.5,-825,625,223,17,106,...,1.675000,9.0,52.0,27.0,5.0,0.281,0.477382,0.166667,0.411765,1
20071031INDWAS,20071031,200.0,1.5,105,1,105,-125,229,9,113,...,1.416667,6.0,56.0,45.0,6.0,0.455,0.462963,0.323232,0.300000,1
20071031MEMSAS,20071031,198.0,7.0,99.5,4.0,210,-250,205,-3,101,...,1.605263,8.0,42.0,22.0,5.0,0.253,0.537857,0.183908,0.521739,1
20071031NOPSAC,20071031,193.0,10.0,94.0,1.5,-525,425,194,14,95,...,1.583333,5.0,34.0,18.0,3.0,0.222,0.506073,0.185185,0.300000,1


In [38]:
ratings_dict = {'itemID': list(window_data.team),
                'userID': list(window_data.opponent),
                'rating': list(window_data.Points)}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is required.
# The Reader class is used to parse a file containing ratings.
reader = Reader(rating_scale=(75, 280))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

In [44]:
# Split data into 5 folds
data.split(n_folds=10)

# svd
algo = SVD()
evaluate(algo, data, measures=['RMSE'])

# nmf
algo = NMF()
evaluate(algo, data, measures=['RMSE'])

Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 4.6912
------------
Fold 2
RMSE: 22.4131
------------
Fold 3
RMSE: 16.2807
------------
Fold 4
RMSE: 4.8591
------------
Fold 5
RMSE: 16.2308
------------
Fold 6
RMSE: 31.3077
------------
Fold 7
RMSE: 12.9885
------------
Fold 8
RMSE: 5.3077
------------
Fold 9
RMSE: 36.6923
------------
Fold 10
RMSE: 24.6923
------------
------------
Mean RMSE: 17.5463
------------
------------
Evaluating RMSE of algorithm NMF.

------------
Fold 1
RMSE: 4.6912
------------
Fold 2
RMSE: 22.3459
------------
Fold 3
RMSE: 16.2807
------------
Fold 4
RMSE: 4.8591
------------
Fold 5
RMSE: 16.2308
------------
Fold 6
RMSE: 31.3077
------------
Fold 7
RMSE: 13.0000
------------
Fold 8
RMSE: 5.3077
------------
Fold 9
RMSE: 36.6923
------------
Fold 10
RMSE: 24.6923
------------
------------
Mean RMSE: 17.5408
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [4.6911559816791994,
                             22.345923068375974,
                             16.280740155164935,
                             4.859126579037754,
                             16.230769230769226,
                             31.30769230769232,
                             13.0,
                             5.307692307692321,
                             36.69230769230768,
                             24.69230769230768]})